In [13]:
#导入需要的包
import pyreadr
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import lightgbm as lgb
from skopt import BayesSearchCV
from xgboost import XGBClassifier
import time


In [ ]:
#读取数据
data1 = pyreadr.read_r("CD.Rdata")
X_train = data1["X_train"]#feature
X_test = data1["X_test"]
y_train = data1["y_train"]#target
#是否有缺失值
sum(pd.isnull(X_train).any())
sum(pd.isnull(X_test ).any())
sum(pd.isnull(y_train).any())

In [ ]:
#读取基于BH方法变量筛选方法处理后的X_train、X_test数据
X_BH= pd.read_csv("X_BH.csv")
X_test_BH = pd.read_csv("X_test_BH.csv")

In [8]:
X_BH= pd.read_csv("D:/生物医疗统计方法/X_BH.csv")
y_train = pd.read_csv("D:/生物医疗统计方法/y_train.csv")

# 基本分类模型和集成学习

## 利用网格搜索进行初步探索

In [16]:
# 初始化 AUC 列表
auc_scores = []

# 循环运行 50 次
for i in range(1,51):
    # 将基因型数据和表现型数据划分为训练集和测试集
    X_train1, X_test1, y_train1, y_test1 = train_test_split(X_BH, y_train, test_size=0.3, random_state=i)
    
    #knn模型
    k_values = list(range(20, 31))  # 设置K值的候选范围
    auc_score_knn = []   # 用于保存每个K值下的交叉验证AUC结果
    for k in k_values:
        clf = KNeighborsClassifier(n_neighbors=k,p = 1,metric = 'minkowski')
        auc = np.mean(cross_val_score(clf, X_train1, y_train1, cv=5, scoring="roc_auc"))
        auc_score_knn.append(auc)  # 在x_train上进行交叉验证，计算每个K值下的AUC
    best_k = k_values[np.argmax(auc_score_knn)]  # 找到表现最佳的K值
    model_knn = KNeighborsClassifier(n_neighbors=best_k,p = 1,metric = 'minkowski')  # 创建KNN模型
    model_knn.fit(X_train1, y_train1)   # 在整个x_train上训练模型
    y_pred = model_knn.predict(X_test1)  # 使用训练好的模型对测试数据进行预测
    auc_scores.append(roc_auc_score(y_test1, y_pred))   # 计算AUC
    
    #logistic regression
    lr = LogisticRegression(random_state=i)
    lr_param_grid = {'C': [0.1, 1.0, 10.0]}
    lr_grid_search = GridSearchCV(lr, lr_param_grid, scoring='roc_auc', cv=5)
    lr_grid_search.fit(X_train1, y_train1)
    lr_best_model = lr_grid_search.best_estimator_
    lr_auc = roc_auc_score(y_test1, lr_best_model.predict_proba(X_test1)[:, 1])
    auc_scores.append(lr_auc)
    
    #Trees
    param = [{'max_depth': np.arange(3,20,3),'min_samples_leaf':np.arange(6,10,2),
              'criterion':['gini','entropy']}]
    tlf = GridSearchCV(DecisionTreeClassifier(random_state=i),param_grid=param,cv=5)  
    # 在x_train上使用网格搜索法选择最佳参数（其中蕴含着交叉验证）
    tlf.fit(X_train1, y_train1)
    model_tree = DecisionTreeClassifier(criterion=tlf.best_params_["criterion"],
                                        max_depth=tlf.best_params_["max_depth"],
                               min_samples_leaf=tlf.best_params_["min_samples_leaf"])
    # 利用局部表现最好的参数值建立决策树分类模型
    model_tree.fit(X_train1, y_train1)
    y_pred = model_tree.predict_proba(X_test1)[:, 1]
    auc_scores.append(roc_auc_score(y_test1, y_pred))
    
    # 随机森林模型
    rf = RandomForestClassifier(random_state=i)
    rf_param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [1, 5, 10]}
    rf_grid_search = GridSearchCV(rf, rf_param_grid, scoring='roc_auc', cv=5)
    rf_grid_search.fit(X_train1, y_train1)
    rf_best_model = rf_grid_search.best_estimator_
    rf_auc = roc_auc_score(y_test1, rf_best_model.predict_proba(X_test1)[:, 1])
    auc_scores.append(rf_auc)


    # GBDT模型
    gbdt = GradientBoostingClassifier(random_state=i)
    gbdt_param_grid = {'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7]}
    gbdt_grid_search = GridSearchCV(gbdt, gbdt_param_grid, scoring='roc_auc', cv=5)
    gbdt_grid_search.fit(X_train1, y_train1)
    gbdt_best_model = gbdt_grid_search.best_estimator_
    gbdt_auc = roc_auc_score(y_test1, gbdt_best_model.predict_proba(X_test1)[:, 1])
    auc_scores.append(gbdt_auc)

    # SVM模型
    svm = SVC(probability=True, random_state=i)
    svm_param_grid = {'C': [0.1, 1.0, 10.0], 'kernel': ['linear', 'rbf']}
    svm_grid_search = GridSearchCV(svm, svm_param_grid, scoring='roc_auc', cv=5)
    svm_grid_search.fit(X_train1, y_train1)
    svm_best_model = svm_grid_search.best_estimator_
    svm_auc =roc_auc_score(y_test1, svm_best_model.predict_proba(X_test1)[:, 1])
    auc_scores.append(svm_auc)

    # LightGBM模型
    lgbm = lgb.LGBMClassifier(random_state=i)
    gbm_param_grid = {'learning_rate': [0.1, 0.01, 0.001], 'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7]}
    gbm_grid_search = GridSearchCV(lgbm, gbm_param_grid, scoring='roc_auc', cv=5)
    gbm_grid_search.fit(X_train1, y_train1)
    gbm_best_model = gbm_grid_search.best_estimator_
    gbm_auc = roc_auc_score(y_test1, gbm_best_model.predict_proba(X_test1)[:, 1])
    auc_scores.append(gbm_auc)
    
    
    #XGBoost
    xgb = XGBClassifier(random_state=i)
    xgb_param_grid = {
        'max_depth': [3, 5, 7],
        'learning_rate': [0.05, 0.1, 0.3],
        'min_child_weight':[3, 4, 5, 6]
    }
    xgb_grid_search = GridSearchCV(xgb, xgb_param_grid, scoring='roc_auc', cv=5)
    xgb_grid_search.fit(X_train1, y_train1)
    xgb_best_model = xgb_grid_search.best_estimator_
    xgb_auc = roc_auc_score(y_test1, xgb_best_model.predict_proba(X_test1)[:, 1])
    auc_scores.append(xgb_auc)
    print("这是循环",i)
    
#将数据放进表格
auc_scores =pd.DataFrame(data=auc_scores)
auc_scores.to_csv('auc_scores.csv')

这是循环 1


## 基于贝叶斯优化进一步调参优化

以下代码由于运算时间问题所以没有写在一个for循环下面，但是通过设置相同的随机种子来控制对于不同方法的训练集和测试集相同

In [ ]:
# lightGBM循环50次
n_iterations = 50
auc_scores_lightGBM_BH = []
time_lightGBM_BH = []

for i in range(n_iterations):
    
    start_time = time.time()
    
    # 划分训练集和测试集
    X_train1, X_test1, y_train1, y_test1 = train_test_split(X_BH, y_train, test_size=0.3, random_state=i)#设置了随机数种子

    # 定义LightGBM分类器
    lgb_classifier = lgb.LGBMClassifier(random_state=i)

    # 定义参数空间
    param_space = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'n_estimators': (50, 500),
        'max_depth': (1, 10),
        'num_leaves': (10, 100),
        'min_child_samples': (1, 20),
        'subsample': (0.1, 1.0, 'uniform'),
        'colsample_bytree': (0.1, 1.0, 'uniform')
    }

    # 使用贝叶斯优化进行参数调优
    optimizer = BayesSearchCV(lgb_classifier, param_space, scoring='roc_auc', cv=3, n_iter=50, n_jobs=-1)
    optimizer.fit(X_train1, y_train1)

    # 输出最优参数
    print(f"Iteration {i+1}: Best params - {optimizer.best_params_}")

    # 在测试集上进行预测
    y_pred = optimizer.predict_proba(X_test1)[:, 1]

    # 计算AUC得分
    auc = roc_auc_score(y_test1, y_pred)
    auc_scores_lightGBM_BH.append(auc)
    #print(f"Iteration {i+1}: AUC - {auc}")
    
    end_time = time.time()
    duration = end_time - start_time
    time_lightGBM_BH.append(duration)
    
    
auc_scores_lightGBM_BH =pd.DataFrame(data=auc_scores_lightGBM_BH)#将数据放进表格
auc_scores_lightGBM_BH.to_csv('auc_scores_lightGBM_BH.csv')
time_lightGBM_BH =pd.DataFrame(data=time_lightGBM_BH)#将数据放进表格
time_lightGBM_BH.to_csv('auc_scores_lightGBM_BH.csv')


In [ ]:
# SVM循环50次
n_iterations = 50
auc_scores_SVM_BH = []
time_SVM_BH = []

for i in range(n_iterations):
        
    start_time = time.time()
    
    # 划分训练集和测试集
    X_train1, X_test1, y_train1, y_test1 = train_test_split(X_BH, y_train, test_size=0.3, random_state=i)#设置了随机数种子


    # 定义SVM分类器
    svm_classifier = SVC(probability=True, random_state=i)

    # 定义参数空间
    param_space = {
        'C': (0.01, 10.0, 'log-uniform'),
        'gamma': (0.01, 10.0, 'log-uniform'),
        'kernel': ('linear', 'rbf')
    }

    # 使用贝叶斯优化进行参数调优
    optimizer = BayesSearchCV(svm_classifier, param_space, scoring='roc_auc', cv=3, n_iter=50, n_jobs=-1)
    optimizer.fit(X_train1, y_train1)

    # 输出最优参数
    print(f"Iteration {i+1}: Best params - {optimizer.best_params_}")

    # 在测试集上进行预测
    y_pred = optimizer.predict_proba(X_test1)[:, 1]

    # 计算AUC得分
    auc = roc_auc_score(y_test1, y_pred)
    auc_scores_SVM_BH.append(auc)
    print(f"Iteration {i+1}: AUC - {auc}")
        
    end_time = time.time()
    duration = end_time - start_time
    time_SVM_BH.append(duration)

auc_scores_SVM_BH =pd.DataFrame(data=auc_scores_SVM_BH)#将数据放进表格
auc_scores_SVM_BH.to_csv('auc_scores_SVM_BH.csv')
time_SVM_BH =pd.DataFrame(data=time_SVM_BH)#将数据放进表格
time_SVM_BH.to_csv('time_SVM_BH.csv')


In [ ]:
# 随机森林循环50次
n_iterations = 50
auc_scores_RF_BH = []
time_RF_BH = []

for i in range(n_iterations):
        
    start_time = time.time()
    
    # 划分训练集和测试集
    X_train1, X_test1, y_train1, y_test1 = train_test_split(X_BH, y_train, test_size=0.3, random_state=i)#设置了随机数种子


    # 定义随机森林分类器
    rf_classifier = RandomForestClassifier(random_state=i)

    # 定义参数空间
    param_space = {
        'n_estimators': (50, 500),
        'max_depth': (1, 10),
        'min_samples_split': (2, 20),
        'min_samples_leaf': (1, 20),
        'max_features': (0.1, 1.0, 'uniform')
    }

    # 使用贝叶斯优化进行参数调优
    optimizer = BayesSearchCV(rf_classifier , param_space, scoring='roc_auc', cv=3, n_iter=50, n_jobs=-1)
    optimizer.fit(X_train1, y_train1)

    # 输出最优参数
    print(f"Iteration {i+1}: Best params - {optimizer.best_params_}")

    # 在测试集上进行预测
    y_pred = optimizer.predict_proba(X_test1)[:, 1]

    # 计算AUC得分
    auc = roc_auc_score(y_test1, y_pred)
    auc_scores_RF_BH.append(auc)
    print(f"Iteration {i+1}: AUC - {auc}")
        
    end_time = time.time()
    duration = end_time - start_time
    time_RF_BH.append(duration)

auc_scores_RF_BH =pd.DataFrame(data=auc_scores_RF_BH)#将数据放进表格
auc_scores_RF_BH.to_csv('auc_scores_RF_BH.csv')
time_RF_BH_BH =pd.DataFrame(data=time_RF_BH)#将数据放进表格
time_RF_BH.to_csv('time_RF_BH.csv')


In [ ]:
# GBDT循环50次
n_iterations = 50
auc_scores_BH_GBDT = []
time_BH_GBDT = []

# 定义参数空间
param_space = {
    'learning_rate': (0.01, 1.0, 'log-uniform'),
    'n_estimators': (50, 500),
    'max_depth': (1, 10),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 20),
    'max_features': (0.1, 1.0, 'uniform')
}


for i in range(n_iterations):
        
    start_time = time.time()
    
    # 划分训练集和测试集
    X_train2, X_test2, y_train2, y_test2 = train_test_split(X_BH, y_train, test_size=0.3, random_state=i)#设置了随机数种子
    
    # 定义GBDT分类器
    gbdt = GradientBoostingClassifier(random_state=i)
    
    # 使用贝叶斯优化进行参数调优
    optimizer = BayesSearchCV(gbdt, param_space, scoring='roc_auc', cv=3, n_iter=50, n_jobs=-1)
    optimizer.fit(X_train2, y_train2)
    
    # 输出最优参数
    print(f"Best params: {optimizer.best_params_}")
    
    # 在测试集上进行预测
    y_pred = optimizer.predict_proba(X_test2)[:, 1]
    
    # 计算AUC得分
    auc = roc_auc_score(y_test2, y_pred)
    auc_scores_BH_GBDT.append(auc)
    
    #print("这是循环",i)
        
    end_time = time.time()
    duration = end_time - start_time
    time_BH_GBDT.append(duration)
    
    

auc_scores_BH_GBDT =pd.DataFrame(data=auc_scores_BH_GBDT)#将数据放进表格
auc_scores_BH_GBDT.to_csv('auc_scores_BH_GBDT.csv')
time_BH_GBDT =pd.DataFrame(data=time_BH_GBDT)#将数据放进表格
time_BH_GBDT.to_csv('time_BH_GBDT.csv')

# 基于GBDT进行预测

In [ ]:
# 定义参数空间
param_space = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'n_estimators': (50, 500),
        'max_depth': (1, 10),
        'num_leaves': (10, 100),
        'min_child_samples': (1, 20),
        'subsample': (0.1, 1.0, 'uniform'),
        'colsample_bytree': (0.1, 1.0, 'uniform')
}

    
# 定义GBDT分类器
lgb = lgb.LGBMClassifier(random_state=42)
    
# 使用贝叶斯优化进行参数调优
optimizer = BayesSearchCV(lgb, param_space, scoring='roc_auc', cv=3, n_iter=50, n_jobs=-1)
optimizer.fit(X_BH, y_train)
    
# 输出最优参数
print(f"Best params: {optimizer.best_params_}")
    
# 在测试集上进行预测
y_pred = optimizer.predict_proba(X_test_BH)[:, 1]
    
y_pred_BH_lightGBT =pd.DataFrame(data=y_pred)#将数据放进表格
y_pred_BH_lightGBT.to_csv('y_pred_BH_lightGBT.csv')

